# Maskin Læring: Klassifisering av dyr

## Trening av vårt første CNN!
![l](https://www.managers.org.uk/wp-content/uploads/2020/03/BrainTraining.jpg)  
I denne notebooken kommer vi til å gjøre et par ting: 
1. Skrive inn studentnummeret vårt
2. Generere et nevralt nettverk(tenk en hjerne) med dette studentnummeret. 
3. Trene dette nettverket

Først så setter vi inn student nummeret vårt.

In [13]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

### La oss nå gå inn på kaggle, hente ut api nøkkelen  vår, og legge den inn under.
### Dette gjør sånn at vi kan bruke datasettet som finnes på kaggle.

In [14]:
kaggle_user= "uzaaft"
kaggle_api = "eef66e4e79f6db8c868a369edc57500c"
os.environ["KAGGLE_USERNAME"] = kaggle_user
os.environ["KAGGLE_KEY"] = kaggle_api
!pip install kaggle
!kaggle datasets download -d alessiocorrado99/animals10

In [15]:
!unzip -q animals10.zip 

In [16]:
import random
Student_nr = input("Hva er student nummeret ditt?")

random.seed(Student_nr)

activation_layers = [
            "relu",
            "sigmoid",
            "softmax",
            "softplus",
            "softsign",
            "tanh",
            "selu",
            "elu",
            "exponential",
]
n = random.randint(1, len(activation_layers)) - 1
layer = activation_layers[n]
lr = random.uniform(1e-4, 1e-1)
print(f"Your learning rate is: {lr}")
print(f"Your activation layer is: {layer}")

In [17]:
base_dir = './raw-img'
LABELS = os.listdir(base_dir)
LABELS.sort()
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant",
                 "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat",
                 "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel",
                 "dog": "cane", "cavallo": "horse", "elephant" : "elefante", 
                 "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto",
                 "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}
LABELS_english = list(translate.values())
base_dir + '/' + LABELS[0]
dataset=[]
testset=[]
count=0

for label in LABELS:
    i=0
#     path = base_dir + '/' + label
    path = os.path.join(base_dir, label)
#     print(path)
    for img in os.listdir(path):
#         print(img)
        image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(75,75))
        image=img_to_array(image)
        image=image/255.0
        if i<1000:
            dataset.append([image,count])
            i+=1
        else:
            testset.append([image,count])
    count=count+1

X,y =zip(*dataset)
test, test_labels=zip(*testset)
y = to_categorical(y)
y=np.array(y)
X=np.array(X)
print("Train Shape:{}\nTrain Labels shape: {}".format(X.shape,y.shape))
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=99)

datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    fill_mode="nearest"
)

In [18]:
def predict_image(image):
    image=img_to_array(image)/255 
    prediction_image=np.array(image)
    prediction_image= np.expand_dims(image, axis=0)
    
    pred = model.predict(prediction_image)
    value = np.argmax(pred)
    italian = LABELS[value]
    print("Prediction : ", LABELS[value])

In [19]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=layer, input_shape=(75, 75, 3), padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(3))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=layer, padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(2))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation=layer, padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(1))


model.add(tf.keras.layers.Conv2D(64, (3, 3), activation=layer, padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(1))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation=layer))
model.add(tf.keras.layers.Dense(32, activation=layer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))



model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
             optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
             metrics=['acc'])

### I cellen ovenfor lager vi modellen. Vi prøver å gjette hvilket dyr det er på bildene med denne ulærte modellen

In [23]:
image=load_img("raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/cavallo/OIP---MGqQIhmz3OEPYP-46_xwHaFj.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/elefante/OIP---LeldVL441fx5S66TGgVQAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/farfalla/OIP--7K5JzW1ZEmrY2YRyVjFCQHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gallina/10.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gatto/1006.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/mucca/OIP---lAIbDlHKmejDpqrXq6vAAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/pecora/OIP--5EOVD9HIiHEqNd8PZjFogHaII.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/ragno/OIP--2QiRmtl6rTZY7AxlnML6gHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/scoiattolo/OIP--0Yr2a_yg2hs5QgCoHAOnAHaLJ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()

In [24]:
history = model.fit(
     datagen.flow(X_train, y_train, batch_size=128), 
    validation_data = (X_valid, y_valid),
    epochs=25
)

### La oss se hvor bra modellen vår gjetter

In [29]:
plt.plot(history.history['acc'], label = "Acc")
plt.plot(history.history['val_acc'], label = "Val Acc")
plt.legend()
plt.show()

In [30]:
plt.plot(history.history['loss'], label = "loss")
plt.plot(history.history['val_loss'], label = "Val loss")
plt.legend()
plt.show()

In [34]:
image=load_img("raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/cavallo/OIP---MGqQIhmz3OEPYP-46_xwHaFj.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/elefante/OIP---LeldVL441fx5S66TGgVQAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/farfalla/OIP--7K5JzW1ZEmrY2YRyVjFCQHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gallina/10.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gatto/1006.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/mucca/OIP---lAIbDlHKmejDpqrXq6vAAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/pecora/OIP--5EOVD9HIiHEqNd8PZjFogHaII.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/ragno/OIP--2QiRmtl6rTZY7AxlnML6gHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/scoiattolo/OIP--0Yr2a_yg2hs5QgCoHAOnAHaLJ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()

In [35]:
history = model.fit(
     datagen.flow(X_train, y_train, batch_size=64), 
    validation_data = (X_valid, y_valid),
    epochs=100, 
    initial_epoch = 25
)

In [36]:
plt.plot(history.history['acc'], label = "Acc")
plt.plot(history.history['val_acc'], label = "Val Acc")
plt.legend()
plt.show()

In [37]:
plt.plot(history.history['loss'],  label = "Loss")
plt.plot(history.history['val_loss'],  label = "Val loss")
plt.legend()
plt.show()

In [39]:
image=load_img("raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/cavallo/OIP---MGqQIhmz3OEPYP-46_xwHaFj.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/elefante/OIP---LeldVL441fx5S66TGgVQAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/farfalla/OIP--7K5JzW1ZEmrY2YRyVjFCQHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gallina/10.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gatto/1006.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/mucca/OIP---lAIbDlHKmejDpqrXq6vAAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/pecora/OIP--5EOVD9HIiHEqNd8PZjFogHaII.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/ragno/OIP--2QiRmtl6rTZY7AxlnML6gHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/scoiattolo/OIP--0Yr2a_yg2hs5QgCoHAOnAHaLJ.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()